In [1]:
from torchvision.datasets import MNIST
import torchvision.transforms as tfs
import pandas as pd
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch.nn.functional import cross_entropy
import torch.nn.functional as F

In [2]:

root = './'
train = MNIST(root, train=True,  transform=tfs.ToTensor(), download=True)
test  = MNIST(root, train=False, transform=tfs.ToTensor(), download=True)


In [3]:
batch_size = 128

train_data = DataLoader(train, batch_size = batch_size, drop_last = True)
test_data = DataLoader(test, batch_size = batch_size, drop_last = True)


In [4]:
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.f_lin = nn.Linear(28**2, 128)
        self.s_lin = nn.Linear(128, 64)
        self.th_lin = nn.Linear(64, 10)
        
    def forward(self, X):
        X = F.relu(self.s_lin(F.relu(self.f_lin(X))))
        return self.th_lin(X)

In [5]:
def convmodel(X, W):
    horrange = 26
    vertrange = 26
    resultpic = torch.FloatTensor(vertrange, horrange).uniform_(-1, 1)
    for i in range(vertrange):
        for j in range(horrange):
            
            horind = j
            vertind = i
            whorind = 0
            wvertind = 0
            summ = 0.
            
            while vertind < i + 3:
                while horind < j + 3:
                    
                    #print(horind)
                    summ += int(W.data[wvertind][whorind] * X[0][vertind][horind])
                    
                    horind+=1
                    whorind+=1
                    
                vertind+=1
                wvertind+=1
                
            resultpic[i][j] = summ
    
    return resultpic
            
            

In [6]:

model = MyModule()


In [23]:
criterion = nn.CrossEntropyLoss()     

optim = torch.optim.Adam(model.parameters(), lr=1e-5)

In [24]:
y_pred = torch.FloatTensor(batch_size)



history = []
epochs = 3

for ind in range(epochs):
    for x_batch, y_batch in train_data:
        
        
# 
        x_batch = x_batch.view(x_batch.shape[0], -1)
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)
        history.append(loss.item())
        optim.zero_grad()
        loss.backward()
        optim.step()
    print(f'{ind+1},\t loss: {history[-1]}')
        

1,	 loss: 0.0016527019906789064
2,	 loss: 0.0015019377460703254
3,	 loss: 0.0013871893752366304


In [15]:
from sklearn.metrics import accuracy_score

In [25]:
acc = 0
batches = 0

for x_batch, y_batch in test_data:
  # загружаем батч данных (вытянутый в линию)
  batches += 1
  x_batch = x_batch.view(x_batch.shape[0], -1)
  y_batch = y_batch

  preds = torch.argmax(model(x_batch), dim=1)
  acc += (preds==y_batch).cpu().numpy().mean()

print(f'Test accuracy {acc / batches:.3}')

Test accuracy 0.98
